In [1]:
import pandas as pd
from pyomo.environ import *

# Load the data
df = pd.read_csv("generators.csv")

df

,Unit,Pmax (MW),Pmin (MW),ai ($/MW2h),bi ($/MWh),ci ($/h),URi (MW/h),DRi (MW/h),Pi (MW),Prohibited Zones
0,G1,500,100,0.0070,7.0,240,80,120,440,"[210,240];[350,380]"
1,G2,200,50,0.0095,10.0,200,50,90,170,"[90,110];[140,160]"
2,G3,300,80,0.0090,8.5,220,65,100,200,"[150,170];[210,240]"
3,G4,150,50,0.0090,11.0,200,50,90,150,"[80,90];[110,120]"
4,G5,200,50,0.0080,10.5,220,50,90,190,"[90,110];[140,150]"
5,G6,120,50,0.0075,12.0,190,50,90,110,"[75,85];[100,105]"


In [2]:
# Create a model
model = ConcreteModel()

# Define sets
model.G = Set(initialize=df.index.tolist())  # Generators

# Define parameters
model.Pmax = Param(model.G, initialize=df['Pmax (MW)'].to_dict())
model.Pmin = Param(model.G, initialize=df['Pmin (MW)'].to_dict())
model.a = Param(model.G, initialize=df['ai ($/MW2h)'].to_dict())
model.b = Param(model.G, initialize=df['bi ($/MWh)'].to_dict())
model.c = Param(model.G, initialize=df['ci ($/h)'].to_dict())
model.UR = Param(model.G, initialize=df['URi (MW/h)'].to_dict())
model.DR = Param(model.G, initialize=df['DRi (MW/h)'].to_dict())
model.demand = Param(initialize=300)  # Total demand (example value)

# Define variables
model.generation = Var(model.G, within=NonNegativeReals)

# Objective function: Minimize total cost
def total_cost(model):
    return sum(model.a[g] * model.generation[g]**2 + model.b[g] * model.generation[g] + model.c[g] for g in model.G)

model.obj = Objective(rule=total_cost, sense=minimize)

# Constraint: Meet the demand
def demand_constraint(model):
    return sum(model.generation[g] for g in model.G) == model.demand

model.demand_constraint = Constraint(rule=demand_constraint)

# Constraint: Generation limits
def generation_limits(model, g):
    return inequality(model.Pmin[g], model.generation[g], model.Pmax[g])

model.generation_limits = Constraint(model.G, rule=generation_limits)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print results
for g in model.G:
    print(f"Generation of {g}: {model.generation[g].value} units")
print(f"Total cost: {model.obj()} units")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.12.13\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Generation of 0: 77.57375805043431 units
Generation of 1: 39.98363155313737 units
Generation of 2: 62.53662428810198 units
Generation of 3: 39.97802219044455 units
Generation of 4: 39.98045549776971 units
Generation of 5: 39.97329523924506 units
Total cost: 4215.310997904946 units


In [4]:
import pandas as pd
from pyomo.environ import *

# Load the data
df = pd.read_csv("generators.csv")

# Create a model
model = ConcreteModel()

# Define sets
model.G = Set(initialize=df.index.tolist())  # Generators

# Define parameters
model.Pmax = Param(model.G, initialize=df['Pmax (MW)'].to_dict())
model.Pmin = Param(model.G, initialize=df['Pmin (MW)'].to_dict())
model.a = Param(model.G, initialize=df['ai ($/MW2h)'].to_dict())
model.b = Param(model.G, initialize=df['bi ($/MWh)'].to_dict())
model.c = Param(model.G, initialize=df['ci ($/h)'].to_dict())
model.UR = Param(model.G, initialize=df['URi (MW/h)'].to_dict())
model.DR = Param(model.G, initialize=df['DRi (MW/h)'].to_dict())
model.demand = Param(initialize=300)  # Total demand (example value)

# Define variables
model.generation = Var(model.G, within=NonNegativeReals)

# Objective function: Minimize total cost
def total_cost(model):
    return sum(model.a[g] * model.generation[g]**2 + model.b[g] * model.generation[g] + model.c[g] for g in model.G)

model.obj = Objective(rule=total_cost, sense=minimize)

# Constraint: Meet the demand with a tolerance
def demand_constraint(model):
    return sum(model.generation[g] for g in model.G) == model.demand

model.demand_constraint = Constraint(rule=demand_constraint)

# Constraint: Generation limits
def generation_limits(model, g):
    return inequality(model.Pmin[g], model.generation[g], model.Pmax[g])

model.generation_limits = Constraint(model.G, rule=generation_limits)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Check solver status
if results.solver.termination_condition != TerminationCondition.optimal:
    print("Warning: Solver did not find an optimal solution.")
    print(f"Termination condition: {results.solver.termination_condition}")
    print(f"Solver message: {results.solver.message}")

# Print results
for g in model.G:
    print(f"Generation of {g}: {model.generation[g].value} units")
print(f"Total cost: {model.obj()} units")

# Additional debug information
total_generation = sum(model.generation[g].value for g in model.G)
print(f"Total generation: {total_generation} units")
print(f"Demand: {model.demand.value} units")

# Check if total generation meets the demand
if total_generation < model.demand.value:
    print("Warning: Total generation is less than the demand.")
elif total_generation > model.demand.value:
    print("Warning: Total generation is more than the demand.")

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot